Group One - Presentation Notebook

**************************  READ ME  *****************************

The first code block in 'Data Preperation' needs to be updated. Update the 'input_path' variable to the directory where you unzipped this notebook's folder. 

For example yours might be something like: C:/users/username/downloads

The raw data is stored within the ism645_flight_delay_analysis/raw_data folder. 

The 'raw_data' folder contains sub-folders with files for RDU flight data from the Bureau of Transportation Statistics (bts) and weather data for RDU from OpenWeather.

******************************************************************

# Introduction

Flight delays are a significant challenge for airport operations, passenger experience, and resource planning. This project combines 2023 historical flight data from the Bureau of Transportation Statistics (BTS) with local weather data from OpenWeather to analyze delay patterns at Raleigh-Durham International Airport (RDU). The objective is to uncover key factors that contribute to delays and develop actionable insights that can enhance operational efficiency and passenger expectations.

### Dataset Overview
The analysis uses two primary datasets:

OpenWeather. History Bulk: Weather Data.

https://openweathermap.org/history-bulk

Bureau of Transportation Statistics. Airline On-Time Performance Data. U.S. Department of Transportation. 

https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ

### Flight Data
Sourced from the Bureau of Transportation Statistics (BTS), this dataset includes operational and delay-related information for 2023 flights. Key variables include:

### Carrier Information:
- `OP_UNIQUE_CARRIER`: Unique identifier for the operating carrier.

### Route Details:
- `ORIGIN`: Origin airport code.
- `DEST`: Destination airport code.
- `DISTANCE`: Flight distance in miles.

### Flight Timing and Delays:
- `DEP_TIME`: Actual departure time.
- `DEP_DELAY_NEW`: Departure delay in minutes (only delays ≥ 0).
- `DEP_DEL15`: Binary flag for 15+ minute departure delay (1 = yes, 0 = no).
- `ARR_TIME`: Actual arrival time.
- `ARR_DELAY_NEW`: Arrival delay in minutes (only delays ≥ 0).
- `ARR_DEL15`: Binary flag for 15+ minute arrival delay (1 = yes, 0 = no).


## Weather Data
Sourced from OpenWeather, this dataset provides detailed meteorological conditions at RDU and surrounding regions, recorded at hourly intervals. Key variables include:

### Temperature Metrics:
- `temp`: Current temperature (°C).
- `temp_min`: Minimum temperature during the period.
- `temp_max`: Maximum temperature during the period.

### Visibility:
- `visibility`: Visibility range in meters.

### Wind Conditions:
- `wind_speed`: Average wind speed (m/s).
- `wind_gust`: Maximum gust speed (m/s).

### Precipitation:
- `rain_1h`: Rain volume in the last hour (mm).
- `rain_3h`: Rain volume in the last 3 hours (mm).
- `snow_1h`: Snow volume in the last hour (mm).
- `snow_3h`: Snow volume in the last 3 hours (mm).

### Cloud Coverage:
- `clouds_all`: Percentage of sky covered by clouds (0-100%).

### Timestamp:
- `dt_est`: Timestamp converted to Eastern Standard Time (EST).

### Research Objectives
The study aims to:
- **Explore Patterns**: Examine how delays vary across time, carriers, and weather conditions.
- **Perform Comparative Analysis**: Assess differences in delay frequencies by season, carrier, and time of day.
- **Develop a Predictive Model**: Leverage flight and weather data to predict delay likelihood and magnitude.

### Analytical Scope and Opportunities
- **Temporal Analysis**: How do delays fluctuate across times of day and seasons?
- **Carrier-Specific Trends**: Are certain airlines more prone to delays, and what operational factors may explain this?
- **Weather Influence**: What is the relationship between weather variables (e.g., wind speed, precipitation) and delays?

### Limitations
This analysis has several limitations:
- **Exclusion of Non-Weather Factors**: The focus is on weather-related data, with other potential delay factors, such as operational or logistical issues, left out.
- **Year-Specific Data**: The dataset covers only 2023, which limits the ability to identify long-term trends or seasonal anomalies.
- **Regional Specificity**: Findings are specific to RDU and may not apply to other airports or regions with different operational contexts.

By combining detailed flight and weather data, this analysis aims to deepen our understanding of delay dynamics and provide actionable recommendations for mitigating delays.


# Data Preperation

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns


# ******************************************************************************************
# *      Update input_path to the directory where you extracted this notebook's folder     *
# ******************************************************************************************
#                                                                                          *
input_path = "C:/Projects"                                                  # **************
#                                                                                          *
# ******************************************************************************************


# list to hold temp dataframes
dataframes = []

# Files are named 01 (for Jan), 02 (for Feb), and so on. The loop iterates through each file, filters for RDU data only, 
# appends data to the temporary list, and concatenates all data into the bts_df variable
# Takes about 15 seconds depending on your device

for i in range(12):
    file_name = f"ism645_flight_delay_analysis/raw_data/bts/{i+1:02}.csv"
    full_path = os.path.join(input_path, file_name)
    
    if i == 0:
        # read in the first file and include the header row
        df = pd.read_csv(full_path)
    else:
        # read in remaining files, skipping the header row
        df = pd.read_csv(full_path, header=0)
        
    # filter for RDU flights only
    rdu_flights = df[(df['ORIGIN'] == 'RDU') | (df['DEST'] == 'RDU')]
    
    dataframes.append(rdu_flights)

# merge temp dataframes - all 2023 flights for RDU
bts_df = pd.concat(dataframes)

Now that we have a dataframe that contains data for all of 2023 specific to arrivals and departures from RDU, we can begin to create some additional variables and prepare the dataset for merger with the weather data. We'll start by adding labels to indicate the name of the airline based on the unique carrier ID. 

In [ ]:
# map the carrier codes to the common airline name
airline_mapping = {
    '9E': 'Endeavor Air',
    'AA': 'American Airlines',
    'AS': 'Alaska Airlines',
    'B6': 'JetBlue Airways Corporation',
    'DL': 'Delta Air Lines, Inc.',
    'F9': 'Frontier Airlines, Inc.',
    'G7': 'GoJet Airlines',
    'MQ': 'Envoy Air',
    'NK': 'Spirit Airlines, Inc.',
    'OH': 'Jetstream Intl',
    'OO': 'SkyWest Airlines',
    'PT': 'Piedmont Airlines',
    'UA': 'United Airlines, Inc.',
    'WN': 'Southwest Airlines',
    'YX': 'Republic Airlines',
    'ZW': 'Air Wisconsin',
    'YV': 'Mesa Airlines, Inc.'
}

# create a new column 'CARRIER_NAME' by mapping carrier codes to airline names
bts_df['CARRIER_NAME'] = bts_df['OP_UNIQUE_CARRIER'].map(airline_mapping)

### account for regional carriers that are subsidiaries of larger airlines ###
# this section maps and adds a 'PARENT_ID' column representing the parent company of regional carriers
bts_df['PARENT_ID'] = bts_df['OP_UNIQUE_CARRIER'].apply(lambda x: 
         'AA' if x in ['AA', 'OO', 'MQ', 'YV', 'PT']  # American Airlines subsidiaries
    else 'DL' if x in ['DL', '9E', 'G7']              # Delta Airlines subsidiaries
    else 'UA' if x in ['UA', 'ZW', 'OH']              # United Airlines subsidiaries
    else x)                                           # for carriers without a parent, retain the original carrier code

# adds the parent carrier name to the df based on PARENT_ID
bts_df['PARENT_CARRIER_NAME'] = bts_df['PARENT_ID'].map(airline_mapping)

In this next step, we create a matchable datetime for each flight using both its flight date and arrival/departure time. The code ensures that the weather data corresponds to the appropriate time, filtered on if the flight is arriving or departing from RDU. The flight times are standardized and merged with the date to create a unified datetime format for comparison with the weather data.

In [ ]:
### extract a matchable datetime from FL_DATE and ARR/DEP_TIME ###

# set arrival and departure times (flt) to Int
bts_df['ARR_TIME'] = pd.to_numeric(bts_df['ARR_TIME'], errors='coerce').astype('Int64')
bts_df['DEP_TIME'] = pd.to_numeric(bts_df['DEP_TIME'], errors='coerce').astype('Int64')

# remove the unnecessary 12:00:00 AM timestamps from the FL_DATE column
bts_df['FL_DATE'] = pd.to_datetime(bts_df['FL_DATE'], format='%m/%d/%Y %I:%M:%S %p')
bts_df['FL_DATE'] = bts_df['FL_DATE'].dt.date

# set WEATHER_TIME to ARR_TIME if the destination is RDU. if the destination is not RDU, set WEATHER_TIME to DEP_TIME.
bts_df['WEATHER_TIME'] = bts_df['ARR_TIME'].where(bts_df['DEST'] == 'RDU', bts_df['DEP_TIME'])

# standardize WEATHER_TIME hour and minute (add leading 0 i.e. 0830)
bts_df['WEATHER_TIME'] = bts_df['WEATHER_TIME'].astype(str).str.zfill(4)

# extract the hour and minute data from WEATHER_TIME
bts_df['WEATHER_HOUR'] = bts_df['WEATHER_TIME'].str[:2]
bts_df['WEATHER_MINUTE'] = bts_df['WEATHER_TIME'].str[2:]

# the corresponding hourly weather data (which is not resolved to the minute like the flight data)
# combine FL_DATE, WEATHER_HOUR, and "00" for the minute value to create a datetime string with the hour floored
# the flight times are floored to the hour to enable all flights within a given hour to be assigned
bts_df['WEATHER_DATETIME'] = bts_df['FL_DATE'].astype(str) + ' ' + bts_df['WEATHER_HOUR'] + ':00'

# convert the combined string to datetime
bts_df['WEATHER_DATETIME'] = pd.to_datetime(bts_df['WEATHER_DATETIME'], errors='coerce')

# drop any NaN WEATHER_DATETIME rows (these likely originate from lack of relevent ARR/DEP_TIME)
bts_df = bts_df.dropna(subset=['WEATHER_DATETIME'])

# sort the dataframe by WEATHER_DATETIME from earliest to latest date
bts_df = bts_df.sort_values(by='WEATHER_DATETIME', ascending=True)

# drop unnecessary columns from bts_df
bts_df = bts_df[['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME',
                       'DEP_DELAY_NEW', 'DEP_DEL15', 'ARR_TIME', 'ARR_DELAY_NEW', 'ARR_DEL15',
                       'DISTANCE', 'CARRIER_NAME', 'PARENT_ID', 'PARENT_CARRIER_NAME', 'WEATHER_DATETIME']]

# drop any duplicate rows
bts_df = bts_df.drop_duplicates()

In this section, the weather data is loaded and cleaned. The dt_iso column is adjusted to remove timezone information and converted to a standard datetime format. We then convert the times from UTC to EST to align with the flight data. The relevant weather columns are retained for further analysis, and any duplicate or missing values in both the weather and flight datasets are removed to ensure data integrity before merging.

In [ ]:
# this block also uses the input_path variable which needed to be updated. it will grab the raw data from within the open_weather folder

raw_weather_data = f"ism645_flight_delay_analysis/raw_data/open_weather/41e77af6ca4e61ed759f49fcee07b86f.csv"
weather_data_path = os.path.join(input_path, raw_weather_data)

weather_df = pd.read_csv(weather_data_path)

# remove the ' +0000 UTC'  info from dt_iso
weather_df['dt_iso_clean'] = weather_df['dt_iso'].str.replace(r'\s\+\d{4}\sUTC$', '', regex=True)

# convert dt_iso to datetime
weather_df['dt'] = pd.to_datetime(weather_df['dt_iso_clean'], utc=True)

# convert utc to est for dt_iso
weather_df['dt_est'] = weather_df['dt'].dt.tz_convert('America/New_York')

# drop duplicates
weather_df = weather_df.drop_duplicates()

# drop unneeded columns from weather_df
weather_df = weather_df[['temp', 
                         'visibility', 'temp_min', 'temp_max', 'wind_speed', 
                         'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h',
                         'clouds_all', 'dt_est']]

Merge the data after removing the unneeded timezone information from the matched columns

In [ ]:
# remove unneeded timezone information from both dataframes
bts_df['WEATHER_DATETIME'] = pd.to_datetime(bts_df['WEATHER_DATETIME']).dt.tz_localize(None)
weather_df['dt_est'] = pd.to_datetime(weather_df['dt_est']).dt.tz_localize(None)

# merge on formatted datetime columns
final_df = pd.merge(
    bts_df,
    weather_df,
    left_on='WEATHER_DATETIME',
    right_on='dt_est',
    how='left'
)

Make some aesthetic and clarity adjustments to column names

In [ ]:
# rename 'WEATHER_DATETIME' to 'DEP_ARR_TIME'
final_df = final_df.rename(columns={'WEATHER_DATETIME': 'DEP_ARR_TIME'})

# convert column names to lowercase because i think it looks nicer
final_df.columns = final_df.columns.str.lower()

Create a categorical 'Season' variable to allow for delay analysis by season

In [ ]:
# maps season ranges for 2023 (including Winter from Jan 1 - Mar 19)
season_ranges = {
    'Winter1': ['2023-01-01', '2023-03-20'],
    'Spring': ['2023-03-20', '2023-06-21'],
    'Summer': ['2023-06-21', '2023-09-23'],
    'Autumn': ['2023-09-23', '2023-12-21'],
    'Winter2': ['2023-12-21', '2024-01-01']
}

# set season_ranges dates to datetime type
season_start_end_dates = {season: pd.to_datetime(dates) for season, dates in season_ranges.items()}

# pass in date to return season according to season_ranges
def assign_season(date):
    for season, (start, end) in season_start_end_dates.items():
        if start <= date < end:
            return season
    return None

# pass dep_arr_time into function and add returned season to season column
final_df['season'] = final_df['dep_arr_time'].apply(assign_season)

# replace Winter1 and Winter2 with Winter
final_df['season'] = final_df['season'].replace({'Winter1': 'Winter', 'Winter2': 'Winter'})

Create a categorical 'Time Of Day' variable to allow for delay analysis by season

In [ ]:
# map time ranges for time of day categories
time_of_day_ranges = {
    'Night1': (0, 6),      # Night1: 00:00 - 05:59
    'Morning': (6, 12),    # Morning: 06:00 - 11:59
    'Afternoon': (12, 18), # Afternoon: 12:00 - 17:59
    'Evening': (18, 21),   # Evening: 18:00 - 20:59
    'Night2': (21, 24)     # Night2: 21:00 - 23:59
}

# pass in 'dep_arr_time' to return the time of day according to time_of_day_ranges
def assign_time_of_day(time):
    for period, (start_hour, end_hour) in time_of_day_ranges.items():
        if start_hour <= time.hour < end_hour:
            return period
    return None  

# pass 'dep_arr_time' to function, and add returned category to 'time_of_day' column
final_df['time_of_day'] = final_df['dep_arr_time'].apply(assign_time_of_day)

# replace Night1 and Night2 with Night
final_df['time_of_day'] = final_df['time_of_day'].replace({'Night1': 'Night', 'Night2': 'Night'})

Create Wind Speed Bins:
The wind_speed variable is categorized into four quantile-based bins: 'Low', 'Medium', 'High', and 'Very High'. This allows for analysis of delays relative to wind speed intensity.

Identify Precipitation Events:
A new column, is_precipitation, is created to flag rows where any precipitation indicator (rain or snow in 1-hour or 3-hour intervals) is present. Rows with any non-NA values in these columns are marked as 1.

Mark Significant Delays:
The has_delay15 column is added to indicate flights delayed by 15 or more minutes. A flight is marked with 1 if either the arrival or departure delay exceeded 15 minutes; otherwise, it is marked as 0.

In [ ]:
# create 'wind_speed' bins relative to quantiles
final_df['wind_speed_bins'] = pd.qcut(final_df['wind_speed'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# mark 1 for is_precipitation if rain or snow in indicator columns
final_df['is_precipitation'] = final_df[['rain_1h', 'rain_3h', 'snow_1h', 'snow_3h']].apply(
    lambda row: 1 if row.notna().any() else 0, axis=1)

# mark 1 for has_delay15 if either arrival or departure was delayed by 15 or more min
final_df['has_delay15'] = ((final_df['arr_del15'] == 1) | (final_df['dep_del15'] == 1)).astype(int)

Create a categorical 'Temperature' variable to allow for delay analysis by temperature ranges

In [ ]:
# map bins and associated labels for temperature catagorical
bins = [-float('inf'), 32, 50, 75, float('inf')]
labels = ['Very Cold', 'Cold', 'Warm', 'Hot']

# create a new column based on these bins
final_df['temp_bins'] = pd.cut(final_df['temp'], bins=bins, labels=labels)

Handle outliers in delay times

In [ ]:
z_cutoff = 3

# create a mask of rows with delayed flights
delayed_mask = final_df['has_delay15'] == 1

# mean and standard deviation for arr_delay_new and dep_delay_new for delayed flights only
arr_delay_mean = np.mean(final_df.loc[delayed_mask, 'arr_delay_new'])
arr_delay_std = np.std(final_df.loc[delayed_mask, 'arr_delay_new'])
dep_delay_mean = np.mean(final_df.loc[delayed_mask, 'dep_delay_new'])
dep_delay_std = np.std(final_df.loc[delayed_mask, 'dep_delay_new'])

# print mean and max values before imputation
print("Before Imputation:")
print(f"Mean arrival delay (arr_delay_new): {final_df['arr_delay_new'].mean():.2f}")
print(f"Mean departure delay (dep_delay_new): {final_df['dep_delay_new'].mean():.2f}")
print(f"Max arrival delay (arr_delay_new): {final_df['arr_delay_new'].max():.2f}")
print(f"Max departure delay (dep_delay_new): {final_df['dep_delay_new'].max():.2f}")

# if the abs(z-score) of the delay time is above z_threshold, replace with NaN
final_df.loc[delayed_mask, 'arr_delay_new'] = final_df.loc[delayed_mask, 'arr_delay_new'].apply(
    lambda x: x if abs((x - arr_delay_mean) / arr_delay_std) < z_cutoff else np.nan)

final_df.loc[delayed_mask, 'dep_delay_new'] = final_df.loc[delayed_mask, 'dep_delay_new'].apply(
    lambda x: x if abs((x - dep_delay_mean) / dep_delay_std) < z_cutoff else np.nan)

# calculate the median of delayed arrivals and departures
arr_delay_median_delayed = final_df.loc[delayed_mask, 'arr_delay_new'].median()
dep_delay_median_delayed = final_df.loc[delayed_mask, 'dep_delay_new'].median()

# impute NaN values with the respective median for arrival or depature flights
final_df['arr_delay_new'] = final_df['arr_delay_new'].fillna(arr_delay_median_delayed)
final_df['dep_delay_new'] = final_df['dep_delay_new'].fillna(dep_delay_median_delayed)

# print mean and max values after imputation
print("\nAfter Imputation:")
print(f"Mean arrival delay (arr_delay_new): {final_df['arr_delay_new'].mean():.2f}")
print(f"Mean departure delay (dep_delay_new): {final_df['dep_delay_new'].mean():.2f}")
print(f"Max arrival delay (arr_delay_new): {final_df['arr_delay_new'].max():.2f}")
print(f"Max departure delay (dep_delay_new): {final_df['dep_delay_new'].max():.2f}")


Finalize the Dataframe structure for analysis

In [ ]:
# map an intuitive column order
column_order = [
    'op_unique_carrier', 'carrier_name', 'parent_carrier_name', 
    'origin', 'dest', 'distance', 'parent_id',
    'dep_time', 'arr_time', 'dep_arr_time', 'time_of_day',
    'dep_delay_new', 'dep_del15', 'arr_delay_new', 'arr_del15','has_delay15',
    'temp', 'temp_min', 'temp_max', 'wind_speed', 'wind_gust',
    'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all',
    'is_precipitation', 'wind_speed_bins', 'temp_bins', 'season'
]

# reorder the columns in final_df
final_df = final_df[column_order]

### optional output the final-df to CSV

# output_path = r'C:\Projects\ism645_flight_delay_analysis\final_flight_data.csv'
# final_df.to_csv(output_path, index=False)

# Data Exploration

In [ ]:
# this block just contains a function to render results in dataframe formats as tables

def render_results_table(result_df, title):
    """
    Renders a dataframe as a table in a matplotlib figure.

    Args:
        result_df (pd.DataFrame): The DataFrame containing results to render.
        title (str): The title for the table.
    """
    # create figure with hight adjusted to number of rows in dataframe
    fig, ax = plt.subplots(figsize=(9, len(result_df) * 0.4))  # Adjust height based on the number of rows
    ax.axis('tight')
    ax.axis('off')

    # Create the table
    table = ax.table(
        cellText = result_df.values, 
        colLabels = result_df.columns,
        rowLabels = result_df.index, 
        cellLoc='center', 
        loc='center'
    )

    # set font size and dynamically adjust column width
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(result_df.columns))))
    
    # set title and show table
    plt.title(title, fontsize=12, pad=10)
    plt.show()

Summary Statistics

This section provides an analysis of the dataset by examining summary statistics for key numerical variables. The code block below generates descriptive statistics for select numerical variables and displays them in a formatted table.

In [ ]:
# map the variable names for the numerical columns
numerical_columns = ['distance', 'dep_delay_new', 'arr_delay_new', 'has_delay15', 'is_precipitation',
                     'temp', 'wind_speed']

# calculate summary statistics for the numerical variables
summary_statistics = final_df[numerical_columns].describe().T

# round the calculated values to 2 decimal places and select desired columns
summary_statistics = summary_statistics.round(2)
summary_statistics = summary_statistics[['mean', 'std', 'min', 'max']]

render_results_table(summary_statistics, "Summary Statistics")

Frequency Analysis

In this section, we analyze the frequency distributions of categorical variables: `parent_carrier_name`, `wind_speed_bins`, `temp_bin`, and `season`. Each variable's frequency is calculated as a percentage of the total and visualized using bar plots.


In [ ]:
frequency_data = {
    'parent_carrier_name': final_df['parent_carrier_name'].value_counts(normalize=True) * 100,
    'wind_speed_bins': final_df['wind_speed_bins'].value_counts(normalize=True) * 100,
    'temp_bins': final_df['temp_bins'].value_counts(normalize=True) * 100,
    'season': final_df['season'].value_counts(normalize=True) * 100
}

# Create a bar plot for each variable in the frequency table
fig, plot_axes = plt.subplots(len(frequency_data), 1, figsize=(7, 20))

for plot_axis, (variable_name, frequency_table) in zip(plot_axes, frequency_data.items()):
    frequency_table.plot(kind='bar', ax=plot_axis, color='royalblue')
    plot_axis.set_title(f"Frequency Distribution of {variable_name}")
    plot_axis.set_xlabel(variable_name)
    plot_axis.set_ylabel("Delay Percentage (%)")
    plot_axis.set_xticklabels(plot_axis.get_xticklabels(), rotation=35, ha='right')

plt.tight_layout()
plt.show()


Correlation Analysis

This section presents a correlation analysis to examine the relationships between numerical variables, including flight delays, weather conditions, and distance. A heatmap visualization is used to present the results, with annotations showing the correlation coefficients for easier interpretation.


In [ ]:
# map desired numerical columns from final_df
corr_columns = [
    'distance', 'dep_delay_new', 'arr_delay_new', 'temp', 'temp_min', 'temp_max',
    'wind_speed', 'wind_gust', 'clouds_all', 'is_precipitation'
]

# create df_corr using only the selected numerical variables
df_corr = final_df[corr_columns]

# calculate the correlations
correlation_matrix = df_corr.corr()

# plot the correlation heatmap
plt.figure(figsize=(12, 5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.2)
plt.title('Correlation Matrix of Flight and Weather Variables')
plt.show()

Statistical Significance Testing

Key questions:

1. Are there significant differences in delay proportions between carriers, season, or time of day?
2. Does the proportion of delays differ significantly between flights that occur during precipitation versus non-precipitation conditions?


In [ ]:
### test for differences in delay times between carriers

# count number of delayed and on_time flights by carrier
carrier_num_delays = (
    final_df.groupby('parent_carrier_name')['has_delay15']
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={0: 'on_time', 1: 'delayed'})
)

# calculate expected values
row_sums = carrier_num_delays.sum(axis=1)
col_sums = carrier_num_delays.sum(axis=0)
total = row_sums.sum()
expected = np.outer(row_sums, col_sums) / total
expected = expected[:, 0:2]

# set up a dataframe for the results
result = pd.DataFrame({
    'On Time': carrier_num_delays['on_time'],
    'Delayed': carrier_num_delays['delayed'],
    'Proportion': (carrier_num_delays['delayed'] / (carrier_num_delays['on_time'] + carrier_num_delays['delayed'])).round(2),
    'Expected On Time': expected[:, 0].round(2),
    'Expected Delayed': expected[:, 1].round(2)
}, index=carrier_num_delays.index)


# calculate chi2 for each airline and p-value using the observed and expected values

# empty lists to store values
chi2_values = []
p_values = []

for i in range(len(result)):
    # observed values for each airline
    observed = carrier_num_delays.iloc[i].values
    expected_vals = expected[i]
    
    # calculate chi2 statistic for each airline
    chi2_stat = np.sum((observed - expected_vals)**2 / expected_vals)
    
    # compute the p-value for the chi2 statistic (df=1 for 2x2 table)
    p_val = 1 - stats.chi2.cdf(chi2_stat, df=1)
    
    # append values to lists
    chi2_values.append(round(chi2_stat, 2))
    p_values.append(round(p_val, 4))

# add chi2 and p-value columns to the result dataframe
result['Chi2'] = chi2_values
result['P-Value'] = p_values

render_results_table(result, "ChiSquare Test for Significant Delay Proportion Differences Between Carriers")


For many of the variables, it makes sense to test whether or not the proportions within each category (i.e. Morning, Afternoon etc in time_of_day) are statistically different from the overall proportion of delays. To enable processing of any desired categorical column, a function is defined below to run the z-test for proportions and return the result.

In [ ]:
# tests for statistically significant difference in proportion of delays for unique categories from provided column against overall frequency of has_delay15

def test_category_deviation_from_overall(categorical_column):

    #calculate the proportion of delays for all flights in 2023 at RDU
    overall_prop = final_df['has_delay15'].mean()

    # initialize a list to store results
    results = []

    # loop through each unique category within the passed in categorical_column (i.e. Spring, Summer....)
    for category, group in final_df.groupby(categorical_column):
        # delay proportion for the provided category
        category_prop = group['has_delay15'].mean()
        # number of observations in current category
        num_obs = group.shape[0]

        # z-score calculation for category delay proportion
        standard_error = (overall_prop * (1 - overall_prop) / num_obs) ** 0.5
        z = (category_prop - overall_prop) / standard_error

        # p-value for the z-test
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))  # Two-tailed test

        # append to results list
        results.append({
            f'{categorical_column}': category,
            f'Proportion': round(category_prop, 2),
            'Overall Proportion': round(overall_prop, 2),
            'Z-Score': round(z, 2),
            'P-Value': round(p_value, 4)
        })

    # convert results to dataframe
    results_df = pd.DataFrame(results)
    
    return results_df


# some examples below - other categories include wind_speed_bins, temp_bin, & time_of_day
results_airlines = test_category_deviation_from_overall('parent_carrier_name')
render_results_table(results_airlines, "Carrier vs Overall Delay Proportion")

results_seasons = test_category_deviation_from_overall('season')
render_results_table(results_seasons, "Season vs Overall Delay Proportion")

In [ ]:
# this codeblock is essentailly the same as the test function but shows the steps a little more clearly
# you can confirm the similarity by looking at the output of passing the is_precipitation column into the test_category_deviation_from_overall function (in the next codeblock)

# Group 1: Precipitation = 1
delays_precip = final_df[(final_df['is_precipitation'] == 1) & (final_df['has_delay15'] == 1)].shape[0]  # Delays with precipitation
total_precip = final_df[final_df['is_precipitation'] == 1].shape[0]  # Total flights with precipitation

# Group 2: Precipitation = 0
delays_no_precip = final_df[(final_df['is_precipitation'] == 0) & (final_df['has_delay15'] == 1)].shape[0]  # Delays without precipitation
total_no_precip = final_df[final_df['is_precipitation'] == 0].shape[0]  # Total flights without precipitation


# calculate the with/without precipitation proportions
p_precip = delays_precip / total_precip
p_no_precip = delays_no_precip / total_no_precip

# calculate the pooled proportion
total_delays = delays_precip + delays_no_precip
total_flights = total_precip + total_no_precip
pooled_prop = total_delays / total_flights


standard_error = np.sqrt(pooled_prop * (1 - pooled_prop) * (1 / total_precip + 1 / total_no_precip))
z_stat = (p_precip - p_no_precip) / standard_error

# perform z-test
p_value = 1 - stats.norm.cdf(z_stat)

# display the results
print(f'Proportion with precipitation: {p_precip:.3f}')
print(f'Proportion without precipitation: {p_no_precip:.3f}')
print(f'Z-statistic: {z_stat:.2f}')
print(f'P-value: {p_value:.4f}')


In [ ]:
# comparison to the above codeblock using the functions to calculate and render the results

render_results_table(test_category_deviation_from_overall('is_precipitation'), "Is Precipitation")